In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
import pathlib

#from py_files.c_target_exploration import mod_concat_NEMDE
#from py_files.d_concat_DUID_marginal_fuelsource import *

## __Loading our file__

In [2]:
file_path = '../data/marginal_emission.pkl'

In [5]:
df = pd.read_pickle(file_path)
df.head()

## __Exercise__

From start to end, the index of our dataframe should increment in 5 min intervals.

__Task 1__:

Write a function which takes in our dataframe "df" and checks if the index of our dataframe entirely increments in 5 min intervals. If your fucntions finds any interval not equal to 5 min, it should return the position where this occurs (e.g. row 783) and duration of the interval at that position (e.g. row 783: 25 min).

__Task 2__:

If you find any such different intervals, modify our dataframe index so that it entirely increments in 5 min intervals. __Hint:__ check out the pd.data_range() function.

__Note__:

Use your knowledge from your pandas lecture and use Google as your best friend.

In [48]:
def correct_timedelta(df, time_diff):

#your code here


    return index_and_duration_of_any_non_5min_time_gap

In [22]:
a1,b1 = correct_timedelta(df_mean_nonans, 300)

In [26]:
b1[:10]

[(Timestamp('2019-07-31 20:25:00'), 600),
 (Timestamp('2019-07-31 19:30:00'), 600),
 (Timestamp('2019-07-31 03:55:00'), 600),
 (Timestamp('2019-07-30 18:40:00'), 600),
 (Timestamp('2019-07-30 05:20:00'), 600),
 (Timestamp('2019-07-29 18:40:00'), 600),
 (Timestamp('2019-07-28 14:45:00'), 600),
 (Timestamp('2019-07-27 12:30:00'), 600),
 (Timestamp('2019-07-27 04:40:00'), 600),
 (Timestamp('2019-07-26 18:10:00'), 600)]

In [7]:
#hypothetical number of 5min intervals given the max and min values of the df time range
h = pd.date_range(start=df.index.min(), end=df.index.max(), freq="5min")
len(h)

1060704